In [6]:
import logging
import numpy as np
import os

import torch
import torch.nn as nn

torch.manual_seed(42)
np.random.seed(42)

from tqdm import tqdm
import torch.optim as optim

In [8]:
from IPython.display import clear_output

!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/model.py
!wget https://raw.githubusercontent.com/artfin/PES-Fitting-MSA/master/dataset.py

from model import FCNet
from dataset import PolyDataset
from dataset import HTOCM

clear_output()

In [9]:
# original github repo
# https://github.com/amirgholami/adahessian
# 
# lessw2020 update
# https://github.com/lessw2020/Best-Deep-Learning-Optimizers/tree/master/adahessian

!wget https://raw.githubusercontent.com/lessw2020/Best-Deep-Learning-Optimizers/master/adahessian/adahessian.py

from adahessian import Adahessian

clear_output()

adahessian.Adahessian